Metrična in deljena metrična dimenzija za splošne grafe

In [4]:
def metric_dimension(G):
    # funkcija vrne metrično dimenzijo grafa G
    n = G.num_verts()  # število vozlišč
    D = G.distance_matrix()  # matrika razdalj med vsakima dvema vozliščema v grafu
    p = MixedIntegerLinearProgram(maximization = False)
    x = p.new_variable(binary = True)  # x_i je 1, če je i-to vozlišče v rešljivi množici, in 0 sicer
    for u, v in Combinations(range(n), 2):
        p.add_constraint(sum(x[w] for w in range(n)  # dobljena vsota je število x_i, pri katerih se razlikuje razdalja do vozlišč iz para?
                             if D[u, w] != D[v, w]) >= 1)  # razlikovati se mora vsaj pri enem paru
    p.set_objective(sum(x[w] for w in range(n)))  # minimiziramo število vozlišč v rešljivi množici
    return p.solve()

In [5]:
def fractional_metric_dimension(G):
    # funkcija vrne deljeno metrično dimenzijo grafa G
    n = G.num_verts()
    D = G.distance_matrix()
    p = MixedIntegerLinearProgram(maximization = False)
    x = p.new_variable(real=True, nonnegative=True)  # najprej zahtevamo x_i >= 0 
    p.set_max(x,1)  # nato pa še x_i <= 1
    for u, v in Combinations(range(n), 2):
        p.add_constraint(sum(x[w] for w in range(n)
                             if D[u, w] != D[v, w]) >= 1)
    p.set_objective(sum(x[w] for w in range(n)))
    return p.solve()

Nekaj primerov izračuna

In [7]:
G = graphs.CompleteGraph(12)
print(metric_dimension(G))
print(fractional_metric_dimension(G))

11.0
6.0


In [9]:
I = graphs.CompleteBipartiteGraph(3,2)
print(metric_dimension(I))
print(fractional_metric_dimension(I))

3.0
2.5


In [11]:
K = graphs.CycleGraph(5)
print(metric_dimension(K))
print(fractional_metric_dimension(K))

2.0
1.25


In [12]:
L = graphs.PathGraph(10)
print(metric_dimension(L))
print(fractional_metric_dimension(L))

1.0
1.0


In [9]:
P = graphs.CompleteGraph(500)
print(fractional_metric_dimension(P))

250.0


In [9]:
generatorji = [graphs.nauty_geng("{0} -c".format(n)) for n in range(1,9)]

razlika = 0
for gen in generatorji:
    for g in gen:
        razlika = max(razlika, metric_dimension(g) - fractional_metric_dimension(g))
    print(razlika)

0
0
0.5
1.0


1.5


2.0


2.5


3.0


Merjenje časovne zahtevnosti

In [3]:
# spodnja zanka je v pomoč pri merjenju časovne zahtevnosti metrične dimenzije za majhne grafe

def zanka_md(gen):
    st_grafov = 0
    for g in gen:
        st_grafov = st_grafov + 1
        metric_dimension(g)
    return st_grafov

generatorji = [graphs.nauty_geng("{0} -c".format(n)) for n in range(1,9)]

for gen in generatorji:
    %time st_grafov = zanka_md(gen)
    print(st_grafov)

CPU times: user 10 ms, sys: 24.7 ms, total: 34.7 ms
Wall time: 114 ms
1
CPU times: user 7.55 ms, sys: 9.45 ms, total: 17 ms
Wall time: 38.3 ms
1
CPU times: user 2.94 ms, sys: 8.83 ms, total: 11.8 ms
Wall time: 23.2 ms
2
CPU times: user 8.33 ms, sys: 8.94 ms, total: 17.3 ms
Wall time: 27.5 ms
6
CPU times: user 35.4 ms, sys: 11.9 ms, total: 47.3 ms
Wall time: 57.7 ms
21


CPU times: user 266 ms, sys: 47.2 ms, total: 313 ms
Wall time: 361 ms
112


CPU times: user 2.35 s, sys: 236 ms, total: 2.58 s
Wall time: 2.67 s
853


In [4]:
# spodnja zanka je v pomoč pri merjenju časovne zahtevnosti deljene metrične dimenzije za majhne grafe

def zanka_fmd(gen):
    st_grafov = 0
    for g in gen:
        st_grafov = st_grafov + 1
        fractional_metric_dimension(g)
    return st_grafov

generatorji = [graphs.nauty_geng("{0} -c".format(n)) for n in range(1,9)]

for gen in generatorji:
    %time st_grafov = zanka_fmd(gen)
    print(st_grafov)

CPU times: user 15.8 ms, sys: 20.6 ms, total: 36.4 ms
Wall time: 94.5 ms
1
CPU times: user 8.93 ms, sys: 8.59 ms, total: 17.5 ms
Wall time: 43.5 ms
1
CPU times: user 4.02 ms, sys: 12.5 ms, total: 16.6 ms
Wall time: 32.1 ms
2
CPU times: user 58 µs, sys: 19.5 ms, total: 19.5 ms
Wall time: 34.6 ms
6
CPU times: user 30.3 ms, sys: 6.8 ms, total: 37.1 ms
Wall time: 54.2 ms
21


CPU times: user 166 ms, sys: 11.6 ms, total: 178 ms
Wall time: 199 ms
112


CPU times: user 1.78 s, sys: 28.2 ms, total: 1.81 s
Wall time: 1.92 s
853


In [4]:
def povprecen_cas_md(n):
    # pomožna funkcija, s katero si pomagamo pri merjenju časovne zahtevnosti metrične dimenzije za večje grafe
    st_grafov = 0
    for i in range(1,21):
        G = graphs.RandomGNP(n, i/20)
        if G.is_connected():
            st_grafov += 1
            metric_dimension(G)
    return st_grafov

for n in [10, 20, 30, 40, 50]:
    %time st_grafov = povprecen_cas_md(n)
    print(st_grafov)

CPU times: user 144 ms, sys: 16.7 ms, total: 161 ms
Wall time: 199 ms
16


CPU times: user 2.13 s, sys: 57.1 ms, total: 2.18 s
Wall time: 2.25 s
17


CPU times: user 46.8 s, sys: 1.88 s, total: 48.7 s
Wall time: 51 s
18


In [7]:
def povprecen_cas_fmd(n):
    # pomožna funkcija, s katero si pomagamo pri merjenju časovne zahtevnosti deljene metrične dimenzije za večje grafe
    st_grafov = 0
    for i in range(1,21):
        G = graphs.RandomGNP(n, i/20)
        if G.is_connected():
            st_grafov += 1
            fractional_metric_dimension(G)
    return st_grafov

for n in [10, 20, 30, 40, 50]:
    %time st_grafov = povprecen_cas_fmd(n)
    print(st_grafov)

CPU times: user 82 ms, sys: 12.3 ms, total: 94.2 ms
Wall time: 83.9 ms
16


CPU times: user 304 ms, sys: 4.05 ms, total: 308 ms
Wall time: 332 ms
17


CPU times: user 1.13 s, sys: 33.2 ms, total: 1.16 s
Wall time: 1.19 s
19


CPU times: user 2.27 s, sys: 65.1 ms, total: 2.33 s
Wall time: 2.71 s
19


CPU times: user 4.92 s, sys: 70.7 ms, total: 5 s
Wall time: 5.16 s
19
